# ML model generation

This notebook creates and trains a simple fully connected model using MNIST dataset. At the end, the model is exported as `model.bin`.

In [ ]:
#|export
import torchvision
from torchvision import transforms
from torch import nn

/home/doyu/mambaforge/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/doyu/mambaforge/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
#|export
def generate_dataloader(batch_size=32):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    
    trainset = torchvision.datasets.MNIST("./data", train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    
    testset = torchvision.datasets.MNIST("./data", train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    return trainloader, testloader

In [ ]:
#|export
def test_model(model, testloader):
    loss_fn = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        vloss = 0.
        correct = 0.
        for X,y in testloader:
            out = model(X)
            vloss += loss_fn(out, y).item()
            correct += (torch.argmax(out, 1)==y).float().sum().item()
    
    return vloss/len(testloader),  correct/len(testloader.dataset)

def train_model(model):  
    # training
    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=0.001)
    trainloader, testloader = generate_dataloader()

    for epoch in range(1):

        model.train()
        tloss = 0
        for X,y in trainloader:
            opt.zero_grad()
            out = model(X)
            loss = loss_fn(out, y)
            loss.backward()
            tloss += loss.item()
            opt.step()

        tloss = tloss/len(trainloader)
        vloss, correct = test_model(model, testloader)

        print('LOSS train {} valid {} accuracy {:.5f}'.format(tloss, vloss, correct))

In [ ]:
import torch
import random
seed = 1337
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
trainloader, testloader = generate_dataloader()

In [ ]:
%%time
from export import export_model
from model import Model

model = Model()
train_model(model)
export_model(model)

LOSS train 0.26354814784054953 valid 0.0891189479771942 accuracy 0.97410
wrote model.bin
CPU times: user 1min, sys: 2.92 s, total: 1min 3s
Wall time: 10.5 s


In [ ]:
loss, acc = test_model(torch.load("model.pt"), testloader)
print(f"Loss: {loss:.4f} Accuracy: {100*acc:.2f} %")

assert loss - 0.0891 < 0.0001
assert acc - 0.97410 < 0.00001

Loss: 0.0891 Accuracy: 97.41 %
